<a href="https://colab.research.google.com/github/JackyYang27/data-science-and-big-data/blob/main/comparing_course_selection_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datascience import *
!pip install matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.font_manager as fm

# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib as mpl
from matplotlib.font_manager import fontManager

fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 學期列表
# semesters = ["1041(01)"]
semesters = ["1021(01)", "1022(01)", "1031(01)", "1032(01)", "1041(01)", "1042(01)", "1051(01)", "1052(01)", "1061(01)", "1062(01)", "1071(01)", "1072(01)"]

# 篩選出特定幾個系的所有年級的資料
departments = ['數學系']
# departments = ['企業管理學系', '資訊管理學系', '經濟學系', '財務金融學系'] #管理學院
# departments = ['文學院學士班', '中國文學系', '英美語文學系', '法國語文學系'] #文學院
# departments = ['理學院學士班', '物理學系', '數學系', '化學學系', '光電科學與工程學系'] #理學院
# departments = ['工學院學士班', '化學工程與材料工程學系', '土木工程學系', '機械工程學系'] #工學院
# departments = ['資訊電機學院學士班', '電機工程學系', '資訊工程學系', '通訊工程學系'] #資電學院
# departments = ['地球科學學院學士班', '地球科學學系', '大氣科學學系', '太空科學與工程學系'] #地科學院
# departments = ['客家語文暨社會科學學系'] #客家學院
# departments = ['生命科學系', '生醫科學與工程學系'] #生醫學院
# departments = [
#     '文學院學士班', '中國文學系', '英美語文學系', '法國語文學系',
#     '理學院學士班', '物理學系', '數學系', '化學學系', '光電科學與工程學系',
#     '工學院學士班', '化學工程與材料工程學系', '土木工程學系', '機械工程學系',
#     '企業管理學系', '資訊管理學系', '財務金融學系', '經濟學系',
#     '資訊電機學院學士班', '電機工程學系', '資訊工程學系', '通訊工程學系',
#     '地球科學學院學士班', '地球科學學系', '大氣科學學系', '太空科學與工程學系',
#     '客家語文暨社會科學學系', '生命科學系', '生醫科學與工程學系'
# ] #全校大學部

# 對每個學期進行分析
for semester in semesters:
    # 讀取學期平均成績資料
    url_1 = f"https://raw.githubusercontent.com/JackyYang27/data-science-and-big-data/main/dataset/112-0004_%E6%A0%A1%E5%8B%99%E6%8B%8B%E8%BD%89%E8%B3%87%E6%96%99_%E5%AD%B8%E7%94%9F%E5%AD%B8%E6%9C%9F%E5%B9%B3%E5%9D%87%E6%88%90%E7%B8%BE_{semester}.csv"
    df_1 = pd.read_csv(url_1)
    df_filtered_1 = df_1[df_1['系所名稱'].isin(departments)]
    df_filtered_1['學期-班一般人數'] = pd.to_numeric(df_filtered_1['學期-班一般人數'], errors='coerce')
    df_filtered_1 = df_filtered_1.dropna(subset=['學期-班一般人數', '學期-班一般名次'])

    # 找出前5%的學生學號和剩餘95%的學生學號
    top_students = pd.DataFrame()
    remaining_students = pd.DataFrame()
    for (dept, year_class), group in df_filtered_1.groupby(['系所名稱', '年級班別']):
        top_n = max(int(round(len(group) * 0.05)), 1)
        top_students_in_group = group.nsmallest(top_n, '學期-班一般名次')
        remaining_students_in_group = group.iloc[top_n:]
        top_students = pd.concat([top_students, top_students_in_group], ignore_index=True)
        remaining_students = pd.concat([remaining_students, remaining_students_in_group], ignore_index=True)

    top_student_ids = top_students['學號'].tolist()
    remaining_student_ids = remaining_students['學號'].tolist()

    # 讀取選課資料
    url_2 = f"https://raw.githubusercontent.com/JackyYang27/data-science-and-big-data/main/dataset/112-0004_%E6%A0%A1%E5%8B%99%E6%8B%8B%E8%BD%89%E8%B3%87%E6%96%99_%E5%AD%B8%E7%94%9F%E9%81%B8%E8%AA%B2%E6%88%90%E7%B8%BE%E8%B3%87%E6%96%99_{semester}.csv"
    df_2 = pd.read_csv(url_2)
    selected_courses_top = df_2[(df_2['學號'].isin(top_student_ids)) & (df_2['課程屬性名稱'].isin(['選修', '通識'])) & (~df_2['分發結果名稱'].isin(['人工退選','停修']))]
    selected_courses_remaining = df_2[(df_2['學號'].isin(remaining_student_ids)) & (df_2['課程屬性名稱'].isin(['選修', '通識'])) & (~df_2['分發結果名稱'].isin(['停修','人工退選']))]

    # 計算每門課程被選擇的次數
    course_counts_top = selected_courses_top['課程名稱'].value_counts()
    course_counts_remaining = selected_courses_remaining['課程名稱'].value_counts()

    # 篩選出同時被前5%和剩餘95%學生選的課程
    common_courses = course_counts_top.index.intersection(course_counts_remaining.index)
    course_counts_top_common = course_counts_top[common_courses]
    course_counts_remaining_common = course_counts_remaining[common_courses]

    # 計算所有前5%學生和剩餘95%學生的總數
    total_top_students = len(top_student_ids)
    total_remaining_students = len(remaining_student_ids)

    # 計算比例
    course_proportions_top_common = course_counts_top_common / total_top_students
    course_proportions_remaining_common = course_counts_remaining_common / total_remaining_students

    # # # 計算比例 #去除comon
    # # course_proportions_top = course_counts_top / total_top_students
    # # course_proportions_remaining = course_counts_remaining / total_remaining_students

    # import pandas as pd
    # from scipy.stats import chi2_contingency

    # # 建立列聯表 #去除comon
    # data = {'Top 5%': course_counts_top_common, 'Remaining 95%': course_counts_remaining_common}
    # contingency_table = pd.DataFrame(data)

    # # 進行卡方檢定
    # chi2, p, dof, expected = chi2_contingency(contingency_table)


    # # 輸出結果
    # print(f"Chi-Square Value = {chi2}")
    # print(f"P-Value = {p}")

    # 繪製柱狀圖
    fig, ax = plt.subplots(figsize=(12, 6))
    course_proportions_top_common.plot(kind='bar', color='blue', position=0, width=0.4, label='Top 5% Students')
    course_proportions_remaining_common.plot(kind='bar', color='green', position=1, width=0.4, label='Remaining 95% Students')
    plt.xlabel('課程名稱')
    plt.ylabel('選課比例')
    plt.title(f'前5%與剩餘95%學生選課比例比較 - {semester}')
    plt.xticks(rotation=90)
    plt.legend()
    plt.tight_layout()
    plt.show()